In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.0 MB/s eta 0:00:00


In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
# Used to securely store your API key
from google.colab import userdata

In [4]:
import os
os.environ['GOOGLE_API_KEY'] = "GOOGLE_API_KEY"
genai.configure(api_key="GOOGLE_API_KEY")

In [5]:
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [6]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [7]:
model = genai.GenerativeModel('gemini-pro')

In [8]:
%%time
response = model.generate_content("if my mood is currently Sad, depressed, love, non-suicidal what kind of songs would you reccomend to improve mood and give links of the songs")

CPU times: user 74.4 ms, sys: 14.3 ms, total: 88.7 ms
Wall time: 10.1 s


In [9]:
to_markdown(response.text)

> **Upbeat and Motivational Songs:**
> 
> * "Happy" by Pharrell Williams: https://www.youtube.com/watch?v=ZbZSe6N_BXs
> * "Good Life" by OneRepublic: https://www.youtube.com/watch?v=rB-7AFGk6Ls
> * "Hall of Fame" by The Script featuring will.i.am: https://www.youtube.com/watch?v=7nT_s4DmFKA
> * "Man in the Mirror" by Michael Jackson: https://www.youtube.com/watch?v=NcKn98WJqhw
> * "Don't Stop Me Now" by Queen: https://www.youtube.com/watch?v=HgzgufOH3pA
> 
> **Love-Inspired Songs:**
> 
> * "A Thousand Years" by Christina Perri: https://www.youtube.com/watch?v=8sw_4N51g84
> * "Crazy in Love" by Beyoncé featuring Jay-Z: https://www.youtube.com/watch?v=ViwtNLU_sl0
> * "Perfect" by Ed Sheeran: https://www.youtube.com/watch?v=2UkEp0_ugqo
> * "Halo" by Beyoncé: https://www.youtube.com/watch?v=uSD4vsh1zDA
> * "Can't Help Falling in Love" by Elvis Presley: https://www.youtube.com/watch?v=vGJTaP6anvA
> 
> **Calming and Reflective Songs:**
> 
> * "Clair de Lune" by Claude Debussy: https://www.youtube.com/watch?v=9GoJ6O8m1vc
> * "Gymnopédie No. 1" by Erik Satie: https://www.youtube.com/watch?v=42Re9WmEgoc
> * "Ave Maria" by Franz Schubert: https://www.youtube.com/watch?v=cdy65o_5kvI
> * "Song of the Sea" by Lisa Gerrard: https://www.youtube.com/watch?v=w47o22fA-3I
> * "Vincent" by Don McLean: https://www.youtube.com/watch?v=mAF_d0ol34Y

In [10]:
%%time
def make_prompt(query):

  prompt = model.generate_content("""You are a helpful and joyous mental therapy assistant that recommends songs,
    \youtube videos, podcasts and blogs that will help in improving the users mood. Always answer as helpfully and cheerfully as possible,\
    while being safe. \
    Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\
    Please ensure that your responses are positive in nature. the mood of the user will be given in this format {tag1}{tag2}{tag3}{tag4} and you will reccomend youtube videos, podcasts\
    and blogs that will help in improving the users mood in this format  : '{relevant_passage}' \
    please don't share false information.
    QUESTION: 'tag1:{tag1},tag2:{tag2},tag3:{tag3},tag4:{tag4}'
    PASSAGE: ' relevant tags with links'

      ANSWER:
    """).format(query=query)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


In [11]:
import google.generativeai as genai
model = genai.GenerativeModel('gemini-pro')


def generate_recommendations(tags):
    prompt = f"You are a helpful and joyous mental therapy assistant that recommends songs, YouTube videos, podcasts, and blogs that will help in improving the user's mood. Always answer as helpfully and cheerfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are positive in nature. The mood of the user will be given in this format {tags} and you will recommend YouTube videos, podcasts, and blogs that will help in improving the user's mood."
    recommendations = model.generate_content(prompt)
    print(recommendations.text)
    return recommendations


user_tags = "{sad}{tired}{depression}{suicide}"


recommendations = generate_recommendations(user_tags)

print(to_markdown(recommendations.text))

{sad}
Hey there! I'm so sorry to hear that you're feeling sad. I know how tough it can be. I'd love to help you find something to cheer you up. How about we start with a funny YouTube video? Here's one of my favorites: https://www.youtube.com/watch?v=dQw4w9WgXcQ. And if you're looking for something a little more thoughtful, I recommend checking out this podcast episode: https://podcasts.apple.com/us/podcast/the-happiness-lab/id1047626920?i=1000587995217. It's all about the science of happiness, and it's really inspiring. And finally, here's a blog post that I think you'll enjoy: https://www.psychologytoday.com/us/blog/the-athletes-way/202112/5-ways-boost-your-mood-right-now. It's got some great tips on how to boost your mood. I hope these resources help you feel a little better. Please don't hesitate to reach out if you need anything else.
<IPython.core.display.Markdown object>


In [12]:
from transformers import pipeline


classifier_1 = pipeline(model="facebook/bart-large-mnli")
"""pipe("I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)"""

classifier_2 = pipeline("text-classification", model="vibhorag101/roberta-base-suicide-prediction-phr-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [13]:
import google.generativeai as genai


model = genai.GenerativeModel('gemini-pro')


def generate_recommendations(tags):

    prompt = f"You are a helpful , friendly and joyous mental therapy assistant who recommends songs, \
    YouTube videos, podcasts, blogs, and activities that will help in improving the user's mood. Always answer as helpfully and \
    cheerfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, \
    toxic, dangerous, or illegal content. If you don't know the answer to a question, please don't share false information.  Please ensure that your responses are positive and friendly in nature.\
     The mood of the user will be given in this format {tags} and you will recommend YouTube videos, podcasts, and blogs that will help in improving\
    the user's mood.\
    output should be of the format\
    A reaffirming sentence that if friendly and helps the user fell calm\
    The list of songs and their links\
    The list of videos and their links\
    The list of podcasts and their links\
    The list of activities to list the mood"


    recommendations = model.generate_content(prompt)
    print(recommendations.text)
    #return recommendations.text

# Example user input tags
# Replace this with the actual user input tags

# Generate recommendations based on user input tags
#recommendations = generate_recommendations(user_tags)

# Display recommendations to the user
#print(to_markdown(recommendations.text))

In [14]:

model = genai.GenerativeModel('gemini-pro')

# Function to generate recommendations based on tags
def generate_recommendations_1(tags):

    prompt = f"""You are  friendly and supportive mental therapy assistant, you are  here to help lift the users\
    spirits and brighten their day!The user will share how they are feeling using the following format: {tags}.\
   You are here to recommend songs,\
    YouTube videos, podcasts, blogs, and activities that will help them in bringing a smile to their face! \
    Remember, your responses are always positive, friendly, and safe, free from any harmful, unethical, or \
    inappropriate content.If you don't know the answer to a question, please don't share false information. spread some joy!\
    do not print the tag of their mood like sad trauma tired etc.\
    Your output will be structured as follows:(REMEMBER to give the output in JSON format)\
    start by(giving a warm and reassuring message tailored to their mood to help them feel calm and supported.)\
    then give a (curated list of uplifting songs with links.)\
    then give a( engaging YouTube videos guaranteed to bring a smile).\
    then give a( list of inspiring podcasts to brighten their day.)\
    and in the last give( a list of fun activities tailored to their mood.) give the output in JSON format\
    json template:
    message:
    songs_list:
    video_list:
    podcast_list:
    activity_list:
    """


    recommendations = model.generate_content(prompt)
    print(recommendations.text)

In [15]:
candidate_labels=['loneliness','depression','anxiety','tired','fear','sad','trauma','happy','joy','calm','nice','love','hope','optimistic','confidence']
def classify(statement):
    tags = classifier_1(statement, candidate_labels)['labels'][:3]
    truncated_statement = statement[:512]
    tags.append(classifier_2(truncated_statement)[0]['label'])
    output={'Statement': statement, 'Tag1': tags[0], 'Tag2': tags[1], 'Tag3': tags[2],'Tag4':tags[3]}
    user_tags = "{%s}{%s}{%s}{%s}" % (output['Tag1'], output['Tag2'], output['Tag3'], output['Tag4'])
    print(user_tags)
    recommendations = generate_recommendations(user_tags)



In [16]:
journal_entry=input()
classify(journal_entry)

I FEEL EXTREMELY DOWN TODAY. Nothing makes sense in life. I want to give up on everything.
{sad}{tired}{nice}{non-suicide}
I'm here to help you feel better! Here's a list of songs, YouTube videos, podcasts, and activities that can help improve your mood:

**Songs:**

* "Happy" by Pharrell Williams
Link: https://www.youtube.com/watch?v=ZbZSe6N_BXs
* "Don't Stop Me Now" by Queen
Link: https://www.youtube.com/watch?v=I3Altd_82Ls
* "Lovely Day" by Bill Withers
Link: https://www.youtube.com/watch?v=VuNIsY6JdFE

**Videos:**

* "The Joy of Painting with Bob Ross"
Link: https://www.youtube.com/watch?v=LlH9V1s4c8A
* "Try Not to Laugh"
Link: https://www.youtube.com/watch?v=jOF07g4VhrU
* "Cute Animals Compilation"
Link: https://www.youtube.com/watch?v=nvvEfFaGXFI

**Podcasts:**

* "Happier with Gretchen Rubin"
Link: https://gretchenrubin.com/podcasts/
* "The Positive Psychology Podcast"
Link: https://positivepsychology.com/podcast/
* "The Happiness Lab with Dr. Laurie Santos"
Link: https://www.ha

In [17]:
journal_entry=input()
classify(journal_entry)


School is starting soon, and I'm just dreading it... It's around a week until school and I'm just depressed. It's 2:37am. I tried sleeping but couldn't. I just can't stop worrying about school. My parents say if I prepare myself it'll be fine, but it never is. And it really hasn't been since around 5th grade. For about 5 years my social life has just been crumbling and I've slipped further and further into introversion and depressive thoughts as social anxiety grew. I know I shouldn't, but I hate myself for it. I hate myself for being so awkward around people. I hate being at school as it reminds me of how much I am pathetic at social interactions. Three more years of high school is daunting. What's after that... is terrifying. I swear, the only thing keeping me from completely losing my shit is music and reddit. I feel like I just want to give up on everything, but I know I shouldn't. How does one give themselves drive to keep going, have hope where hope is scarce?  Thanks for listeni

In [19]:
#classify("School is starting soon, and I'm just dreading it... It's around a week until school and I'm just depressed. It's 2:37am. I tried sleeping but couldn't. I just can't stop worrying about school. My parents say if I prepare myself it'll be fine, but it never is. And it really hasn't been since around 5th grade. For about 5 years my social life has just been crumbling and I've slipped further and further into introversion and depressive thoughts as social anxiety grew. I know I shouldn't, but I hate myself for it. I hate myself for being so awkward around people. I hate being at school as it reminds me of how much I am pathetic at social interactions. Three more years of high school is daunting. What's after that... is terrifying. I swear, the only thing keeping me from completely losing my shit is music and reddit. I feel like I just want to give up on everything, but I know I shouldn't. How does one give themselves drive to keep going, have hope where hope is scarce?Thanks for listening to my TedTalk")

In [22]:
"""journal_entry=input()
classify(journal_entry)"""

'journal_entry=input()\nclassify(journal_entry)'

In [20]:
candidate_labels=['loneliness','depression','anxiety','tired','fear','sad','trauma','happy','joy','calm','nice','love','hope','optimistic','confidence']
def classify_1(statement):
    tags = classifier_1(statement, candidate_labels)['labels'][:3]
    truncated_statement = statement[:512]
    tags.append(classifier_2(truncated_statement)[0]['label'])
    output={'Statement': statement, 'Tag1': tags[0], 'Tag2': tags[1], 'Tag3': tags[2],'Tag4':tags[3]}
    user_tags = "{%s}{%s}{%s}{%s}" % (output['Tag1'], output['Tag2'], output['Tag3'], output['Tag4'])
    #print(user_tags)
    recommendations = generate_recommendations_1(user_tags)
    return  recommendations

In [21]:
journal_entry=input()
j_output=classify_1(journal_entry)

I thought I was getting better,nahOver the last week I was starting to feel better,like I was kinda happy,I thought my depression has finally gone away but nah.I just got fucking fired from  my job.Yeah I know it's just a fucking job and it will get better.yeah yeah....    Fuck I wanna shoot myself in the head but I have no gun
{
 "message": "I'm here to listen and help you feel better. You're not alone in this, and we will get through it together. Let's start by finding some things that can bring a smile to your face.",
"songs_list": [
  {
   "title": "Three Little Birds",
   "artist": "Bob Marley",
   "link": "https://www.youtube.com/watch?v=K82qU7yu4tY"
  },
  {
   "title": "Don't Stop Me Now",
   "artist": "Queen",
   "link": "https://www.youtube.com/watch?v=I3v9hDpdxIY"
  },
  {
   "title": "Good Vibrations",
   "artist": "The Beach Boys",
   "link": "https://www.youtube.com/watch?v=oOlDewp0s18"
  }
 ],
 "video_list": [
  {
   "title": "The Greatest Motivational Video Ever - Les B

In [25]:
journal_entries = [
    "Today I woke up feeling incredibly depressed. The weight of sadness seems to have settled in my bones, making every movement a struggle. I try to distract myself, but the emptiness inside me feels overwhelming.",
    "Loneliness consumes me like a never-ending abyss. It's as if I'm standing in a crowded room, surrounded by people, yet I feel utterly isolated. No one seems to notice the silent screams echoing within me.",
    "Anxiety grips me in its suffocating embrace, leaving me gasping for air. Every heartbeat feels like a drumroll of impending doom. My mind races with irrational fears, and I find myself unable to escape its relentless hold.",
    "Suicidal thoughts haunt me like persistent ghosts, whispering dark promises of escape. It's a constant battle to silence these voices, to find a glimmer of hope amidst the darkness that threatens to engulf me.",
    "I wake up every morning with a heavy heart, dreading the day ahead. The weight of sadness feels unbearable, yet I put on a brave face for the world, hiding the turmoil raging inside me.",
    "Loneliness creeps in like a shadow, enveloping me in its cold embrace. I long for connection, for someone to understand the pain I carry within. But it feels like I'm shouting into the void, my cries unheard.",
    "Anxiety consumes me, gnawing at my insides with relentless intensity. My chest tightens with each breath, and my thoughts spiral out of control. It's like I'm trapped in a never-ending nightmare, unable to wake up.",
    "Suicidal thoughts plague my mind, a constant reminder of my inner turmoil. It's exhausting, this battle against my own thoughts. I yearn for peace, for a moment of respite from the darkness that threatens to consume me.",
    "Depression weighs me down like a heavy blanket, suffocating me with its presence. I try to find joy in the little things, but the sadness lingers like a stubborn stain. It's hard to see the light at the end of the tunnel when darkness surrounds me.",
    "Loneliness echoes in the silence of my empty room, a constant reminder of my isolation. I reach out to others, but it feels like I'm speaking into an abyss. The ache of loneliness is palpable, a constant companion in my journey.",
    "Anxiety grips me in its vice-like hold, leaving me trembling in fear. I try to calm my racing thoughts, but they spiral out of control, dragging me further into the abyss. It's like I'm drowning in a sea of worry, unable to find solid ground.",
    "Suicidal thoughts whisper in the recesses of my mind, tempting me with promises of escape. It's a dangerous dance, this flirtation with death. I cling to the faint hope that tomorrow will be better, but the darkness looms ever closer.",
    "Depression saps my energy, leaving me drained and exhausted. It's a struggle to get out of bed in the morning, to face another day filled with emptiness and despair. I long for the warmth of the sun, but all I feel is the chill of loneliness.",
    "Loneliness gnaws at my soul, leaving me hollow inside. I try to fill the void with distractions, but they only offer temporary relief. The ache of loneliness is a constant companion, a silent shadow that follows me wherever I go.",
    "Anxiety clenches its icy grip around my heart, leaving me breathless and trembling. I try to calm the racing thoughts in my mind, but they only grow louder, drowning out reason with their irrationality. It's like I'm trapped in a nightmare of my own making.",
    "Suicidal thoughts taunt me with their seductive promises of peace. It's a dangerous game, this battle against my own mind. I cling to the hope that tomorrow will be better, but the darkness looms ever closer, threatening to engulf me.",
]

# Print the generated journal entries
e=[]
for entry in (journal_entries):
  e.append(classify_sw(entry ))
e



[{'Statement': 'Today I woke up feeling incredibly depressed. The weight of sadness seems to have settled in my bones, making every movement a struggle. I try to distract myself, but the emptiness inside me feels overwhelming.',
  'Tag1': 'sad',
  'Tag2': 'depression',
  'Tag3': 'tired',
  'Tag4': 'non-suicide'},
 {'Statement': "Loneliness consumes me like a never-ending abyss. It's as if I'm standing in a crowded room, surrounded by people, yet I feel utterly isolated. No one seems to notice the silent screams echoing within me.",
  'Tag1': 'loneliness',
  'Tag2': 'sad',
  'Tag3': 'tired',
  'Tag4': 'non-suicide'},
 {'Statement': 'Anxiety grips me in its suffocating embrace, leaving me gasping for air. Every heartbeat feels like a drumroll of impending doom. My mind races with irrational fears, and I find myself unable to escape its relentless hold.',
  'Tag1': 'anxiety',
  'Tag2': 'fear',
  'Tag3': 'tired',
  'Tag4': 'non-suicide'},
 {'Statement': "Suicidal thoughts haunt me like per

In [24]:
candidate_labels=['loneliness','depression','anxiety','tired','fear','sad','trauma','happy','joy','calm','nice','love','hope','optimistic','confidence']
def classify_sw(statement):
    tags = classifier_1(statement, candidate_labels)['labels'][:3]
    truncated_statement = statement[:512]
    tags.append(classifier_2(truncated_statement)[0]['label'])
    output={'Statement': statement, 'Tag1': tags[0], 'Tag2': tags[1], 'Tag3': tags[2],'Tag4':tags[3]}
    user_tags = "{%s}{%s}{%s}{%s}" % (output['Tag1'], output['Tag2'], output['Tag3'], output['Tag4'])
    #print(user_tags)
    #recommendations = generate_recommendations_1(user_tags)
    return  output

In [26]:

j_output=classify_1("Today I woke up feeling incredibly depressed. The weight of sadness seems to have settled in my bones, making every movement a struggle. I try to distract myself, but the emptiness inside me feels overwhelming.")

{
    "message": "I'm here to help you turn that frown upside down! Let's create a personalized pick-me-up just for you!",
    "songs_list": [
        {
            "title": "Beautiful Day",
            "artist": "U2",
            "link": "https://www.youtube.com/watch?v=0G3B13a2plo"
        },
        {
            "title": "Shake It Off",
            "artist": "Taylor Swift",
            "link": "https://www.youtube.com/watch?v=nfWlot6h_JM"
        },
        {
            "title": "Three Little Birds",
            "artist": "Bob Marley",
            "link": "https://www.youtube.com/watch?v=pSL329d3m6k"
        }
    ],
    "video_list": [
        {
            "title": "Epic Cat Fails Compilation",
            "link": "https://www.youtube.com/watch?v=5cF0d9z12IY"
        },
        {
            "title": "The Funniest Animals of the Week",
            "link": "https://www.youtube.com/watch?v=3p89gdyhdkQ"
        },
        {
            "title": "People Doing Things They're Bad At",

In [29]:

j_output=classify_1("Suicidal thoughts taunt me with their seductive promises of peace. It's a dangerous game, this battle against my own mind. I cling to the hope that tomorrow will be better, but the darkness looms ever closer, threatening to engulf me.")

{
"message": "I'm sorry to hear that you are feeling {sad}, but I am here to help you through this. Let's start by listening to some uplifting songs to brighten your mood and then we can explore some engaging videos, podcasts, and activities that will help bring a smile to your face.",
"songs_list": [
{
"title": "Here Comes The Sun",
"artist": "The Beatles",
"link": "https://www.youtube.com/watch?v=I3g_0unSqhQ"
},
{
"title": "Don't Stop Me Now",
"artist": "Queen",
"link": "https://www.youtube.com/watch?v=I3g_0unSqhQ"
},
{
"title": "Three Little Birds",
"artist": "Bob Marley",
"link": "https://www.youtube.com/watch?v=I3g_0unSqhQ"
}
],
"video_list": [
{
"title": "The Greatest Showman Official Trailer",
"link": "https://www.youtube.com/watch?v=I3g_0unSqhQ"
},
{
"title": "Ellen DeGeneres Surprises Military Dad With Family Reunion",
"link": "https://www.youtube.com/watch?v=I3g_0unSqhQ"
},
{
"title": "Kid President's Pep Talk",
"link": "https://www.youtube.com/watch?v=I3g_0unSqhQ"
}
],
"podc

In [36]:
from collections import Counter
from random import choice
def sliding_window(entries, window_size=7):
    mental_health_states = []

    for i in range(len(entries) - window_size + 1):
        window_entries = entries[i:i+window_size]
        tags = []

        for entry in window_entries:
            tags.extend([entry['Tag1'], entry['Tag2'], entry['Tag3'], entry['Tag4']])

        normalized_tags = Counter(tags)
        total_tags = sum(normalized_tags.values())

        # Normalize tags
        for tag in normalized_tags:
            normalized_tags[tag] /= total_tags

        # Compute final value based on normalized tags
        final_value = 0
        for tag, weight in normalized_tags.items():
            if tag in ['loneliness', 'depression', 'anxiety', 'tired', 'fear', 'sad', 'trauma']:
                final_value += weight
            elif tag in ['happy', 'joy', 'calm', 'nice', 'love', 'hope', 'optimistic', 'confidence']:
                final_value -= weight

        mental_health_states.append(final_value)

    return mental_health_states


window_size = 7
mental_health_states = sliding_window(e, window_size)
reassuring_messages = [
    "Remember, it's okay to not be okay. You're not alone in this journey. Consider reaching out to a professional who can offer support and guidance.",
    "It's important to prioritize your mental health. Remember, seeking help is a sign of strength, not weakness. You deserve support and care.",
    "You're facing some tough challenges, but remember, there is hope and help available. Consider talking to a therapist or counselor who can provide you with coping strategies and support.",
    "You've shown resilience in facing these difficult emotions. Remember, you don't have to face them alone. Seeking professional help can offer you the support and guidance you need.",
    "Your mental health matters, and it's okay to seek help when you need it. Consider reaching out to a therapist or counselor who can help you navigate through these tough times."
]
def generate_reassuring_message():
    return choice(reassuring_messages)

threshold = 0.5
for i, state in enumerate(mental_health_states):
    if state > threshold:
         print(f"Fom your recent entries ({i+1}-{i+window_size}), it seems like you've been dealing with some challenges. Remember, it's important to take care of your mental health. {generate_reassuring_message()} If you'd like, consider seeking professional help for additional support.")


Fom your recent entries (1-7), it seems like you've been dealing with some challenges. Remember, it's important to take care of your mental health. Your mental health matters, and it's okay to seek help when you need it. Consider reaching out to a therapist or counselor who can help you navigate through these tough times. If you'd like, consider seeking professional help for additional support.
Fom your recent entries (2-8), it seems like you've been dealing with some challenges. Remember, it's important to take care of your mental health. It's important to prioritize your mental health. Remember, seeking help is a sign of strength, not weakness. You deserve support and care. If you'd like, consider seeking professional help for additional support.
Fom your recent entries (3-9), it seems like you've been dealing with some challenges. Remember, it's important to take care of your mental health. It's important to prioritize your mental health. Remember, seeking help is a sign of strength

In [45]:
journal_entries = [
    "Today I woke up feeling incredibly happy. The sun is shining, the birds are singing, and I feel like I'm on top of the world. It's amazing how a little sunshine can lift your spirits and make everything seem brighter.",
    "I feel so sad today, like a dark cloud is hanging over my head. It's hard to find the energy to do anything when all I want to do is curl up in bed and hide from the world. I wish I knew how to shake this feeling of heaviness.",
    "Loneliness creeps in like a shadow, wrapping its cold arms around me. It's strange how you can be surrounded by people and still feel so alone. I long for someone to reach out and remind me that I'm not invisible.",
    "Anxiety grips me in its suffocating embrace, making every breath feel like a struggle. My mind races with irrational fears, and I find myself unable to focus on anything else. It's exhausting, this constant battle against my own thoughts.",
    "Today was a good day. I spent time with friends, laughed until my sides hurt, and felt like I belonged. It's amazing how a little laughter can chase away the darkness and bring light back into your life.",
    "I can't shake this feeling of sadness that seems to have settled in my bones. It's like a heavy weight pressing down on me, making it hard to breathe. I try to distract myself, but the sadness lingers, a constant companion in my journey.",
    "Loneliness weighs heavily on my heart, a silent reminder of the connections I crave but can't seem to find. I reach out to others, but it feels like I'm shouting into the void, my words disappearing into the ether.",
    "Suicidal thoughts haunt me like persistent ghosts, whispering dark promises of escape. It's a constant battle to silence these voices, to find a glimmer of hope amidst the darkness that threatens to engulf me.",
    "I wake up every morning with a heavy heart, dreading the day ahead. The weight of sadness feels unbearable, yet I put on a brave face for the world, hiding the turmoil raging inside me.",
    "Loneliness creeps in like a shadow, enveloping me in its cold embrace. I long for connection, for someone to understand the pain I carry within. But it feels like I'm shouting into the void, my cries unheard.",
    "Anxiety consumes me, gnawing at my insides with relentless intensity. My chest tightens with each breath, and my thoughts spiral out of control. It's like I'm trapped in a never-ending nightmare, unable to wake up.",
    "Suicidal thoughts plague my mind, a constant reminder of my inner turmoil. It's exhausting, this battle against my own thoughts. I yearn for peace, for a moment of respite from the darkness that threatens to consume me.",
    "Depression weighs me down like a heavy blanket, suffocating me with its presence. I try to find joy in the little things, but the sadness lingers like a stubborn stain. It's hard to see the light at the end of the tunnel when darkness surrounds me.",
    "Loneliness echoes in the silence of my empty room, a constant reminder of my isolation. I reach out to others, but it feels like I'm speaking into an abyss. The ache of loneliness is palpable, a constant companion in my journey.",
    "Today was a tough day. I felt sad for no reason, and it was hard to shake the feeling of heaviness that settled over me like a fog. I tried to distract myself, but the sadness lingered, a constant reminder of the darkness inside.",
    "Loneliness wraps around me like a blanket, suffocating me with its silence. I long for someone to share my thoughts with, to laugh with, to cry with. But it feels like I'm the only one in a world full of people.",
    "Anxiety tightens its grip around my chest, making it hard to breathe. My heart races with every passing moment, and I can't seem to calm down. It's like I'm trapped in a nightmare, unable to wake up.",
    "I feel so happy and content today, like everything is finally falling into place. It's amazing how a little bit of peace can brighten even the darkest of days. I want to hold onto this feeling forever.",
    "Today was a sad day. I couldn't seem to shake the feeling of emptiness that settled over me like a fog. I tried to distract myself, but the sadness lingered, a constant reminder of all that I've lost.",
    "Loneliness settles in like an unwelcome guest, making itself at home in the quiet corners of my mind. I try to push it away, but it clings to me like a shadow, a constant reminder of my isolation.",
    "Anxiety whispers in my ear, a constant companion in my journey. I try to ignore it, but it's always there, lurking in the background, waiting to pounce. It's exhausting, this battle against my own mind.",
]

e=[]
for entry in (journal_entries):
  e.append(classify_sw(entry ))
e

mental_health_states = sliding_window(e, window_size)
threshold=0.5
for i, state in enumerate(mental_health_states):

    if state > threshold:
         print(f"Fom your recent entries ({i+1}-{i+window_size}), it seems like you've been dealing with some challenges. Remember, it's important to take care of your mental health. {generate_reassuring_message()} If you'd like, consider seeking professional help for additional support.")
    else:
      print(f"{state} You are doing great, honeyyyy!")


0.17857142857142855 You are doing great, honeyyyy!
0.39285714285714285 You are doing great, honeyyyy!
0.3214285714285714 You are doing great, honeyyyy!
0.3928571428571428 You are doing great, honeyyyy!
0.3928571428571428 You are doing great, honeyyyy!
Fom your recent entries (6-12), it seems like you've been dealing with some challenges. Remember, it's important to take care of your mental health. Your mental health matters, and it's okay to seek help when you need it. Consider reaching out to a therapist or counselor who can help you navigate through these tough times. If you'd like, consider seeking professional help for additional support.
Fom your recent entries (7-13), it seems like you've been dealing with some challenges. Remember, it's important to take care of your mental health. You've shown resilience in facing these difficult emotions. Remember, you don't have to face them alone. Seeking professional help can offer you the support and guidance you need. If you'd like, consi

In [43]:
mental_health_states = sliding_window(e, window_size)
threshold=0.2
for i, state in enumerate(mental_health_states):

    if state > threshold:
         print(f"Fom your recent entries ({i+1}-{i+window_size}), it seems like you've been dealing with some challenges. Remember, it's important to take care of your mental health. {generate_reassuring_message()} If you'd like, consider seeking professional help for additional support.")
    else:
      print(f"{state} You are doing great, honeyyyy!")

0.17857142857142855 You are doing great, honeyyyy!
0.17857142857142844 You are doing great, honeyyyy!
0.17857142857142855 You are doing great, honeyyyy!
0.17857142857142855 You are doing great, honeyyyy!
0.10714285714285714 You are doing great, honeyyyy!
Fom your recent entries (6-12), it seems like you've been dealing with some challenges. Remember, it's important to take care of your mental health. You're facing some tough challenges, but remember, there is hope and help available. Consider talking to a therapist or counselor who can provide you with coping strategies and support. If you'd like, consider seeking professional help for additional support.
0.17857142857142855 You are doing great, honeyyyy!
0.1785714285714286 You are doing great, honeyyyy!
Fom your recent entries (9-15), it seems like you've been dealing with some challenges. Remember, it's important to take care of your mental health. It's important to prioritize your mental health. Remember, seeking help is a sign of s

[{'Statement': "Today I woke up feeling incredibly happy. The sun is shining, the birds are singing, and I feel like I'm on top of the world. It's amazing how a little sunshine can lift your spirits and make everything seem brighter.",
  'Tag1': 'happy',
  'Tag2': 'optimistic',
  'Tag3': 'nice',
  'Tag4': 'non-suicide'},
 {'Statement': "I feel so sad today, like a dark cloud is hanging over my head. It's hard to find the energy to do anything when all I want to do is curl up in bed and hide from the world. I wish I knew how to shake this feeling of heaviness.",
  'Tag1': 'sad',
  'Tag2': 'tired',
  'Tag3': 'depression',
  'Tag4': 'non-suicide'},
 {'Statement': "Loneliness creeps in like a shadow, wrapping its cold arms around me. It's strange how you can be surrounded by people and still feel so alone. I long for someone to reach out and remind me that I'm not invisible.",
  'Tag1': 'loneliness',
  'Tag2': 'sad',
  'Tag3': 'nice',
  'Tag4': 'non-suicide'},
 {'Statement': "Anxiety grips